#### Conda env : kyuwon_video_swin_transformer(Python 3.10.11)
---
#### [주의사항]
##### Image Size가 (224,224,3)인 경우 Model Parameter를 아래와 같이 고정할 것
##### model = SwinTRansformer3D(patch_size=(4,4,4), embed_dim=96)


In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
import logging
import torch
import torch.nn as nn
from einops import rearrange, reduce
from tqdm import tqdm
import gc
import copy


# 사용하고자 하는 model
# from Video_Swin_Transformer.Video_Swin_Transformer.model import SwinTransformer3D

# R2plus1d_Video_Swin_Transformer
from Video_Swin_Transformer.R2plus1d_Video_Swin_Transformer.model import SwinTransformer3D

/home/ssrlab/anaconda3/envs/kyuwon_video_swin_transformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batchsz = 8
num_workers = 4
epochs = 120
start_epoch = 0
lr = 3e-5

In [3]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:1' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

True
학습을 진행하는 기기: cuda:1


In [4]:
with open('/home/ssrlab/kw/개성형성/video_swin_transformer/Data_Processing/15Frame/Full-Shot/training_set.dat', "rb") as training_file:
    train_input_temp = pickle.load(training_file)
with open('/home/ssrlab/kw/개성형성/video_swin_transformer/Data_Processing/15Frame/Full-Shot/validation_set.dat', "rb") as training_file:
    valid_input_temp = pickle.load(training_file)
    
# 15 Frame / Full-Frame / 224x224x3  : /home/ssrlab/kw/개성형성/video_swin_transformer/Data_Processing/15Frame/Full-Shot/training_set.dat , /home/ssrlab/kw/개성형성/video_swin_transformer/Data_Processing/15Frame/Full-Shot/validation_set.dat
# 15 Frame / Face / 224x224x3 : 
# 15 Frame / Face / 128x128x3 : 

In [5]:
class ChalearnDataset(Dataset):
    def __init__(self,imagedata,tagdata,transform=None):
        self.imagedata=imagedata
        self.tagdata=tagdata
        self.transform = transform
        
    def __len__(self):
        return len(self.imagedata)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_data=self.imagedata[idx]
        image_data=torch.FloatTensor(image_data)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces)
        return image_data,big_five_sorces

In [6]:
train_dataset = ChalearnDataset(imagedata=train_input_temp[0],tagdata=train_input_temp[1])
val_dataset = ChalearnDataset(imagedata=valid_input_temp[0],tagdata=valid_input_temp[1])
# test_dataset = ChalearnDataset(imagedata=test_set_data,tagdata=test_y,transform=transform)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batchsz, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batchsz, shuffle=True, num_workers=num_workers)
# test_dataloader = DataLoader(dataset=test_dataset, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
max_value=0

# 모델 로드

In [7]:
model=SwinTransformer3D(patch_size=(4,4,4), embed_dim=96)

model.to(device)

criterion = torch.nn.L1Loss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

R2plus1d_pretrained_model


/home/ssrlab/anaconda3/envs/kyuwon_video_swin_transformer/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
# checkpoint=torch.load("saved_model/model_0.pth", map_location=device)
# model.load_state_dict(checkpoint["model"])
# optimizer.load_state_dict(checkpoint["optimizer"])
# start_epoch = checkpoint['epoch']

# 학습

In [9]:
# torch.set_printoptions(profile="full")

with torch.cuda.device(1):
    
    trainingEpoch_loss = []
    validationEpoch_loss = []
    
    for i in range(120):
        train_avg_loss = 0
        val_avg_loss = 0
        
        model.train()
        for image_data, big_five_data in tqdm(train_dataloader):
            
            image_data = rearrange(image_data, 'b d h w c -> b c d h w')
            image_data = image_data.to(device)
            
            big_five_data = reduce(big_five_data,'b c d -> b c', 'max')
            big_five_data = big_five_data.to(device)
            
            optimizer.zero_grad()
            hypothesis = model(image_data)
            
            loss = criterion(hypothesis, big_five_data)
            
            loss.backward()
            optimizer.step()
            
            train_avg_loss += loss
        train_avg_loss=train_avg_loss/len(train_dataloader)
        trainingEpoch_loss.append(train_avg_loss)
        print('Epoch = {}, 1 - train_loss = {}'.format(i+1,(1 - train_avg_loss)))
        
        
        with torch.no_grad():
            model.eval()
            for image_data, big_five_data in tqdm(val_dataloader):
                
                image_data = rearrange(image_data, 'b d h w c -> b c d h w')
                image_data = image_data.to(device)
                
                big_five_data = reduce(big_five_data,'b c d -> b c', 'max')
                big_five_data = big_five_data.to(device)
                
                hypothesis = model(image_data)
                
                val_loss = criterion(hypothesis, big_five_data)
                val_avg_loss += val_loss
                
            val_avg_loss=val_avg_loss/len(val_dataloader)
            validationEpoch_loss.append(val_avg_loss)
        print('Epoch = {}, 1 - val_loss = {}'.format(i+1,(1 - val_avg_loss)))
        print('\n')
        
        start_epoch+=1
    
        # torch.save({
        #     'epoch': i+1,
        #     'model': model.state_dict(),
        #     'optimizer': optimizer.state_dict(),
        #     'loss': val_avg_loss,
        # }, save_model_file_path.format('model',start_epoch,'pth'))
        
        torch.cuda.empty_cache()
        gc.collect()

  0%|          | 0/748 [00:00<?, ?it/s]

100%|██████████| 748/748 [05:11<00:00,  2.40it/s]


Epoch = 1, 1 - train_loss = 0.8771129250526428


100%|██████████| 250/250 [00:32<00:00,  7.78it/s]


Epoch = 1, 1 - val_loss = 0.8928838968276978




...

Epoch = 117, 1 - train_loss = 0.9875699877738953


100%|██████████| 250/250 [00:37<00:00,  6.72it/s]


Epoch = 117, 1 - val_loss = 0.9116429090499878




 40%|████      | 302/748 [1:49:43<8:14:11, 66.48s/it]

: 

# 결과 시각화

In [ ]:
temp_1 = []
temp_2 = []
for i in range(len(trainingEpoch_loss)):
    temp_1.append(trainingEpoch_loss[i].item())
    temp_2.append(validationEpoch_loss[i].item())

In [ ]:
from matplotlib import pyplot as plt

plt.clf()
plt.plot(temp_1, label='train_loss')
plt.plot(temp_2, label='val_loss')
# plt.title('')
plt.xlabel('Epoch')
plt.ylabel('1 - MAE')
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot as plt

plt.clf()
plt.plot(temp_2, label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('1 - MAE')
plt.legend()
plt.show()